In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import logging

import src
from src.bert.dataset import PBertDataset
from src.bert.dataset.strategies import MLMin1PopIdeol

In [ ]:
logging.set_verbosity_error()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
THRESHOLDS = {0: 0.415961, 1: 0.295400, 2: 0.429109, 3: 0.302714}
print(DEVICE)

In [ ]:
EXCLUDE_CODERS = []
test = PBertDataset.from_disk(
    path=src.PATH / "data/labeled_data/test.csv.zip",
    label_strategy=MLMin1PopIdeol(),
    exclude_coders=EXCLUDE_CODERS,
)

In [ ]:
COMMIT_HASH = "cf44004e90045cde298e28605ff105747d58aa7a"

tokenizer = AutoTokenizer.from_pretrained("luerhard/PopBERT", revision=COMMIT_HASH)
model = AutoModelForSequenceClassification.from_pretrained(
    "luerhard/PopBERT", revision=COMMIT_HASH
).to(DEVICE)

In [ ]:
collate_fn = test.create_collate_fn(tokenizer)
test_loader = DataLoader(test, collate_fn=collate_fn, batch_size=64, shuffle=False)

In [ ]:
def apply_thresh(y_proba, thresholds: dict):
    y_proba = y_proba.copy()
    for dim, thresh in thresholds.items():
        y_proba[:, dim] = np.where(y_proba[:, dim] > thresh, 1, 0)
    return y_proba

In [ ]:
with torch.inference_mode():
    y_true = []
    y_pred = []
    for batch in test_loader:
        encodings = batch["encodings"]
        encodings = encodings.to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        out = model(**encodings)
        preds = torch.nn.functional.sigmoid(out.logits)
        y_true.extend(batch["labels"].numpy())
        y_pred.extend(preds.cpu().numpy())
    y_pred_05 = np.where(np.array(y_pred) > 0.5, 1, 0)
    y_pred_thresh = apply_thresh(np.array(y_pred), THRESHOLDS)
    y_true = np.array(y_true)

In [ ]:
print(
    classification_report(
        y_true,
        y_pred_05,
        target_names=["elite", "pplcentr", "left", "right"],
        zero_division=0,
    )
)

In [ ]:
print(
    classification_report(
        y_true,
        y_pred_thresh,
        target_names=["elite", "pplcentr", "left", "right"],
        zero_division=0,
    )
)

In [ ]:
performance = classification_report(
    y_true,
    y_pred_thresh,
    target_names=["elite", "pplcentr", "left", "right"],
    zero_division=0,
    output_dict=True,
)

In [ ]:
out = pd.DataFrame(performance).T.drop("support", axis=1)
out = out.reset_index().rename(
    {"index": "Dimension", "precision": "Precision", "f1-score": "F1", "recall": "Recall"},
    axis=1,
)
out["Dimension"] = out["Dimension"].replace(
    {
        "elite": "Anti-Elitism",
        "pplcentr": "People-Centrism",
        "left": "Left-Wing Ideology",
        "right": "Right-Wing Ideology",
    }
)

out = out.iloc[:6, :].round(2)

In [ ]:
out.to_latex(src.PATH / "results/tables/model_performance.tex", index=False)